#06 아이템 기반 최근접 이웃 협업 필터링 실습

##데이터 가공 및 변환

In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv('/content/drive/MyDrive/ESAA(22-1)/Week11/movielens/movies.csv')
ratings = pd.read_csv('/content/drive/MyDrive/ESAA(22-1)/Week11/movielens/ratings.csv')

In [3]:
print(movies.shape)
print(ratings.shape)

(9125, 3)
(100004, 4)


In [6]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [7]:
ratings.describe()

,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,1.000040e+05
mean,347.011310,12548.664363,3.543608,1.129639e+09
std,195.163838,26369.198969,1.058064,1.916858e+08
min,1.000000,1.000000,0.500000,7.896520e+08
25%,182.000000,1028.000000,3.000000,9.658478e+08
50%,367.000000,2406.500000,4.000000,1.110422e+09
75%,520.000000,5418.000000,4.000000,1.296192e+09
max,671.000000,163949.000000,5.000000,1.476641e+09


ratings.csv : 사용자별로 영화에 대한 평점을 매긴 데이터 세트  
평점은 최소 0.5점에서 5점 사이이며 0.5단위로 평점이 부여된다.

협업 필터링은 ratings.csv 데이터 세트와 같이 사용자와 아이템 간의 평점에 기반해 추천하는 시스템이다. 먼저 로우(행) 레벨 형태의 원본 데이터 세트를 다음 모든 사용자를 로우로, 모든 영화를 칼럼으로 구성한 데이터 세트로 변경한다.

In [8]:
ratings = ratings[['userId','movieId','rating']]
ratings_matrix = ratings.pivot_table('rating',index='userId',columns='movieId')
ratings_matrix.head(3)

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


NaN 값이 많은 이유는 사용자가 평점을 매기지 않은 영화가 칼럼으로 변환되면서 NaN 으로 값이 할당되었기 때문이다. 가독성을 높이기 위해 칼럼명을 movieId 가 아닌 영화명 title 로 변경하겠다.

In [11]:
#title 칼럼을 얻기 위해 movies 와 조인
rating_movies = pd.merge(ratings,movies,on='movieId')

#columns='title'로 title 칼럼으로 피벗 수행
ratings_matrix = rating_movies.pivot_table('rating',index='userId',columns='title')

In [12]:
#NaN 값을 모두 0 으로 변환
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix.head(3)

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##영화 간 유사도 산출

cosine_similarity() 함수는 행을 기준으로 서로 다른 행을 비교해 유사도를 산출한다. ratings_matrix 는 userId 가 기준인 행 레벨 데이터이므로 여기에 cosine_similarity()를 적용하면 영화 간의 유사도가 아닌 사용자 간의 유사도가 만들어진다.

영화를 기준으로 cosine_similarity() 를 적용하려면 현재의 ratings_matrix 가 행 기준이 영화가 되고 열 기준이 사용자가 되어야 한다.

In [13]:
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T.head(3)

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
"""Great Performances"" Cats (1998)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
$9.99 (2008),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
from sklearn.metrics.pairwise import cosine_similarity

item_sim = cosine_similarity(ratings_matrix_T,ratings_matrix_T)

#cosine_similarity()로 반환된 넘파이 행렬을 영화명으로 매핑해 DataFrame 으로 변환
item_sim_df = pd.DataFrame(data=item_sim,index=ratings_matrix.columns,columns=ratings_matrix.columns)

print(item_sim_df.shape)
item_sim_df.head(3)

(9064, 9064)


title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
title,,,,,,,,,,,,,,,,,,,,,
"""Great Performances"" Cats (1998)",1.0,0.0,0.0,0.164399,0.020391,0.0,0.014046,0.000000,0.0,0.003166,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
$9.99 (2008),0.0,1.0,0.0,0.000000,0.000000,0.0,0.000000,0.079474,0.0,0.156330,...,0.0,0.0,0.0,0.0,0.0,0.013899,0.0,0.058218,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,1.0,0.000000,0.000000,1.0,0.000000,0.217357,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0


영화 '대부'와 유사도가 높은 상위 5개 영화를 추출해 보겠다.

In [16]:
item_sim_df['Godfather, The (1972)'].sort_values(ascending=False)[:6]

title
Godfather, The (1972)                        1.000000
Godfather: Part II, The (1974)               0.773685
Goodfellas (1990)                            0.620349
One Flew Over the Cuckoo's Nest (1975)       0.568244
American Beauty (1999)                       0.557997
Star Wars: Episode IV - A New Hope (1977)    0.546750
Name: Godfather, The (1972), dtype: float64

기준 영화인 '대부'를 제외하면 '대부-2편'이 가장 유사도가 높다. 장르가 완전히 다른 영화도 유사도가 매우 높게 나타났다.

'인셉션'과 유사도가 높은 영화를 찾아보자.

In [17]:
item_sim_df['Inception (2010)'].sort_values(ascending=False)[1:6]

title
Dark Knight, The (2008)          0.656999
Avatar (2009)                    0.633687
Dark Knight Rises, The (2012)    0.589308
Inglourious Basterds (2009)      0.580237
District 9 (2009)                0.576054
Name: Inception (2010), dtype: float64

'다크나이트'가 가장 유사도가 높다.그 뒤를 이어서 주로 스릴러와 액션이 가미된 좋은 영화가 높은 유사도를 나타내고 있다. 만들어진 아이템기반 유사도 데이터는 사용자의 평점 정보를 모두 취합해 영화에 따라 유사한 다른 영화를 추천할 수 있게 해준다.

##아이템 기반 최근접 이웃 협업 필터링으로 개인화된 영화 추천

앞 예제에서 만든 아이템 기반의 영화 유사도 데이터는 모든 사용자의 평점을 기준으로 영화의 유사도를 생성했고, 개인적인 취향을 반영하지 않고 영화 간의 유사도만을 가지고 추천한 것이다. 

이번 절에서는 영화 유사도 데이터를 이용해 최근접 이웃 협업 필터링으로 개인에게 최적화된 영화 추천을 구현해 보겠다. 개인화된 영화 추천의 가장 큰 특징은 개인이 아직 관람하지 않은 영화를 추천한다는 것이다. 아직 관란함지 않은 영화에 대해서 아이템 유사도와 기존에 관람한 영화의 평점 데이터를 기반으로 해 새롭게 모든 영화의 예측 평점을 계산한 후 높은 예측 평점을 가진 영화를 추천하는 방식이다.

사용자별 영화 예측 평점 R 은 사용자 u의 모든 영화에 대한 실제 평점과 영화 i 의 다른 모든 영화와의 코사인 유사도를 벡터 내적 곱(dot)한 값을 정규화를 위해 |S| 의 합으로 나눈 것을 의미한다.

item_sim_df : 영화 간의 유사도를 가지는 데이터프레임  (R)  
ratings_matrix : 사용자-영화 평점 데이터프레임  (S)  
이용해서 사용자별로 최적화된 평점 스코어를 예측하는 함수를 만든다.

In [18]:
def predict_rating(ratings_arr,item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr)/np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

In [19]:
ratings_pred = predict_rating(ratings_matrix.values,item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred,index=ratings_matrix.index,columns=ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.014679,0.012177,0.008457,0.155729,0.004635,0.008457,0.012224,0.011612,0.011952,0.007415,...,0.020867,0.005313,0.004119,0.012122,0.029437,0.009230,0.000000,0.015098,0.004838,0.000000
2,0.034046,0.032670,0.175621,0.031371,0.014654,0.175621,0.047991,0.057101,0.054119,0.033462,...,0.052653,0.017481,0.022326,0.042259,0.170733,0.045483,0.015784,0.062860,0.005560,0.013151
3,0.008073,0.031750,0.114354,0.035292,0.008948,0.114354,0.025436,0.039356,0.028688,0.044235,...,0.044499,0.017771,0.043463,0.037265,0.218158,0.042527,0.055424,0.042310,0.044989,0.048887


예측 평점이 사용자별 영화의 실제 평점과 영화의 코사인 유사도를 내적(dot)한 값이기 때문에 기존에 영화를 관람하지 않아 0에 해당했던 실제 영화 평점이 예측에서는 값이 부여되는 경우가 많이 발생한다.

사용자가 영화의 평점을 주지 않은 경우 앞에서는 평점을 0으로 부과했다. 개인화된 예측 점수는 평점을 주지 않은 영화에 대해서도 아이템 유사도에 기반해 평점을 예측했다. 따라서 실제와 예측 평점의 차이는 기존에 평점이 부여된 데이터에 대해서만 오차 정도를 측정하겠다.

In [20]:
from sklearn.metrics import mean_squared_error

#사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE를 구함.
def get_mse(pred,actual):
    #평점이 있는 실제 영화만 추출
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred,actual)

print('아이템 기반 모든 최근접 이웃 MSE:',get_mse(ratings_pred,ratings_matrix.values))

아이템 기반 모든 최근접 이웃 MSE: 10.255398119852726


앞의 predict_rating() 함수 : 사용자별 영화의 예측 평점을 계산하기 위해 해당 영화와 다른 모든 영화 간의 유사도 벡터를 적용한 것.  
많은 영화의 유사도 벡터를 이용하다 보니 상대적으로 평점 예측이 떨어졌다. 특정 영화와 가장 비슷한 유사도를 가지는 영화에 대해서만 유사도 벡터를 적용하는 함수로 변경하겠다.   
predict_rating_topsim(ratings_arr,item_sim_arr,N=20) : predict_rating()함수와 유사하지만 N인자를 가지고 있어서 TOP-N 유사도를 가지는 영화 유사도 벡터만 예측값을 계산하는 데 적용한다.

In [24]:
def predict_rating_topsim(ratings_arr,item_sim_arr,n=20):   #ratings_arr : ratings_matrix 에서 실제 영화 평점 / item_sim_arr = item_sim_df에서 아이템별 유사도 값 
    #사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)

    #사용자-아이템 평점 행렬의 열 크기만큼 루프 수행.
    for col in range(ratings_arr.shape[1]):
        #유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 인덱스 반환
        top_n_items = [np.argsort(item_sim_arr[:,col])[:n-1:-1]]
        #개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row,col] = item_sim_arr[col,:][top_n_items].dot(ratings_arr[row,:][top_n_items].T)
            pred[row,col] /= np.sum(np.abs(item_sim_arr[col,:][top_n_items]))
    return pred

In [22]:
ratings_matrix.head(3)   #사용자별 영화 평점

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
item_sim_df.head(3)

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
title,,,,,,,,,,,,,,,,,,,,,
"""Great Performances"" Cats (1998)",1.0,0.0,0.0,0.164399,0.020391,0.0,0.014046,0.000000,0.0,0.003166,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
$9.99 (2008),0.0,1.0,0.0,0.000000,0.000000,0.0,0.000000,0.079474,0.0,0.156330,...,0.0,0.0,0.0,0.0,0.0,0.013899,0.0,0.058218,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,1.0,0.000000,0.000000,1.0,0.000000,0.217357,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0


In [25]:
ratings_pred = predict_rating_topsim(ratings_matrix.values,item_sim_df.values,n=20)
print('아이템 기반 최근접 TOP-20 이웃 MSE:',get_mse(ratings_pred,ratings_matrix.values))

#계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred,index=ratings_matrix.index,columns=ratings_matrix.columns)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if sys.path[0] == '':


아이템 기반 최근접 TOP-20 이웃 MSE: 10.255398119852726


특정 사용자에 대해 영화를 추천해 보겠다.  
userId=9인 사용자에 대해 영화를 추천해 보겠다.

In [26]:
user_rating_id = ratings_matrix.loc[9,:]
user_rating_id[user_rating_id>0].sort_values(ascending=False)[:10]

title
Fargo (1996)                                   5.0
Dead Man Walking (1995)                        5.0
Schindler's List (1993)                        5.0
Shadowlands (1993)                             5.0
Matrix, The (1999)                             5.0
Truman Show, The (1998)                        5.0
As Good as It Gets (1997)                      5.0
William Shakespeare's Romeo + Juliet (1996)    5.0
Sliding Doors (1998)                           4.0
Sixth Sense, The (1999)                        4.0
Name: 9, dtype: float64

'오스틴 파워','반지의 제왕' 등 대작 영화나 어드벤처 영화, 코미디 영화 등 전반적으로 흥행성이 좋은 영화에 높은 평점을 주고 있다.이 사용자에게 아이템 기반 협업 필터링을 통해 영화를 추천하겠다. 먼저 사용자가 이미 평점을 준 영화를 제외하고 추천할 수 있도록 평점을 주지 않은 영화를 리스트 객체로 반환하는 함수인 get_unseen_movies()를 생성한다.

In [28]:
def get_unseen_movies(ratings_matrix,userId):
    #userId로 입력받은 사용자의 모든 영화 정보를 추출해 Series로 반환함.
    #반환된 user_rating은 영화명(title)을 인덱스로 가지는 Series 객체임.
    user_rating = ratings_matrix.loc[userId,:]

    #user_rating이 0보다 크면 기존에 관람한 영화임. 대상 인덱스를 추출해 list 객체로 만듦.
    already_seen = user_rating[user_rating>0].index.tolist()

    #모든 영화명을 list 객체로 만듦
    movies_list = ratings_matrix.columns.tolist()

    #list comprehension으로 already_seen에 해당하는 영화는 movies_list에서 제외함.
    unseen_list = [movie for movie in movies_list if movie not in already_seen]

    return unseen_list

recomm_movie_by_userid() : 사용자가 영화의 평점을 주지 않은 추천 대상 영화 정보와 predict_rating_topsim()에서 추출한 사용자별 아이템 유사도에 기반한 예측 평점 데이터 세트를 이용해 최종적으로 사용자에게 영화를 추천하는 함수

In [29]:
def recomm_movie_by_userid(pred_df,userId,unseen_list,top_n=10):
    #예측 평점 DataFrame에서 사용자id 인덱스와 unseen_list로 들어온 영화명 칼럼을 추출해 가장 예측이 높은 순으로 정렬함.
    recomm_movies = pred_df.loc[userId,unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

#사용자가 관람하지 않는 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix,9)

#아이템 기반의 최근접 이웃 필터링으로 영화 추천
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix,9,unseen_list,top_n=10)

#평점 데이터를 DataFrame으로 생성
recomm_movies = pd.DataFrame(data=recomm_movies.values,index=recomm_movies.index,columns=['pred_df'])

recomm_movies

,pred_df
title,
"6th Man, The (Sixth Man, The) (1997)",0.419065
"Scarlet Letter, The (1926)",0.402807
Frozen Planet (2011),0.299791
"Truce, The (a.k.a. La Tregua) (1996)",0.273464
Family Business (1989),0.251713
Mr. Wonderful (1993),0.251713
"Invisible Circus, The (2001)",0.249181
Patlabor: The Movie (Kidô keisatsu patorebâ: The Movie) (1989),0.242791
"Santa Clause 3: The Escape Clause, The (2006)",0.242521
